In [1]:
import cv2
import os
import time
import uuid

# Creating Image dataset

In [2]:
# Image path
IMAGES_PATH='Tensorflow/workspace/images/test'

In [3]:
# Creating image dataset 15 videos each for 7 sign gestures  
labels=['hello','thanks','yes','no','iloveyou','what','name']
number_imgs=15

In [5]:
# Collecting image dataset for each labels
for label in labels:
    !mkdir {'Tensorflow\workspace\images\collectedimages\\'+label} #Create folder for each label
    cap = cv2.VideoCapture(0)
    print("Collecting images for {}".format(label))
    time.sleep(5)  #take 5s break before collecting new label
    for imgnum in range(number_imgs):
        ret,frame = cap.read()
        imgname = os.path.join(IMAGES_PATH,label,label+'.'+'{}.jpg'.format(str(uuid.uuid1())))
        cv2.imwrite(imgname,frame)
        cv2.imshow('frame',frame)
        time.sleep(2) #take 2s after collecting an image
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
            cap.release()

In [13]:
cap.release()

In [12]:
# No need
cap = cv2.VideoCapture(0)
print("Collecting images for {}".format("hello"))
time.sleep(5)
for imgnum in range(2):
    ret,frame = cap.read()
    imgname = os.path.join(IMAGES_PATH,"hello","hello"+'.'+'{}.jpg'.format(str(uuid.uuid1())))
    cv2.imwrite(imgname,frame)
    cv2.imshow('frame',frame)
    time.sleep(2)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        cap.release()

# Setting up all the required paths

In [39]:
WORKSPACE_PATH = 'Tensorflow/workspace'
SCRIPTS_PATH = 'Tensorflow/scripts'
APIMODEL_PATH = 'Tensorflow/models'
ANNOTATION_PATH = WORKSPACE_PATH+'/annotations'
IMAGE_PATH = WORKSPACE_PATH+'/images'
MODEL_PATH = WORKSPACE_PATH+'/models'
PRETRAINED_MODEL_PATH = WORKSPACE_PATH+'/pre-trained-models'
CONFIG_PATH = MODEL_PATH+'/my_ssd_mobnet/pipeline.config'
CHECKPOINT_PATH = MODEL_PATH+'/my_ssd_mobnet/'

# Creating label paths

In [6]:
# Now will create a pb text file
# The label map needs to be in a .pb text format .so that it can be used by the object detection library 
labels = [
    {'name':'Hello','id':1},
    {'name':'Yes','id':2},
    {'name':'No','id':3},
    {'name':'Thank You','id':4},
    {'name':'I Love You','id':5},
    {'name':'What','id':6},
    {'name':'Name','id':7},
]

with open(ANNOTATION_PATH+'\label_map.pbtxt','w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

In [18]:
# Next will create TF records , TF records are recommended input format for TensorFlow
# Tfrecords are the representation of our data that the object detection api uses
# -x -> XML dir , -l ->labels path , -o ->output directory
!python {SCRIPTS_PATH+'/generate_tfrecord.py'} -x {IMAGE_PATH+'/train'} -l {ANNOTATION_PATH+'/label_map.pbtxt'} -o {ANNOTATION_PATH+'/train.record'}
!python {SCRIPTS_PATH+'/generate_tfrecord.py'} -x {IMAGE_PATH+'/test'} -l {ANNOTATION_PATH+'/label_map.pbtxt'} -o {ANNOTATION_PATH+'/test.record'}

Successfully created the TFRecord file: Tensorflow/workspace/annotations/train.record
Successfully created the TFRecord file: Tensorflow/workspace/annotations/test.record


In [19]:
# Download and extract the TensorFlow model 
!cd Tensorflow && git clone https://github.com/tensorflow/models

Cloning into 'models'...
Updating files:  37% (1149/3046)
Updating files:  38% (1158/3046)
Updating files:  39% (1188/3046)
Updating files:  40% (1219/3046)
Updating files:  41% (1249/3046)
Updating files:  42% (1280/3046)
Updating files:  43% (1310/3046)
Updating files:  44% (1341/3046)
Updating files:  45% (1371/3046)
Updating files:  46% (1402/3046)
Updating files:  47% (1432/3046)
Updating files:  48% (1463/3046)
Updating files:  49% (1493/3046)
Updating files:  50% (1523/3046)
Updating files:  51% (1554/3046)
Updating files:  52% (1584/3046)
Updating files:  53% (1615/3046)
Updating files:  54% (1645/3046)
Updating files:  55% (1676/3046)
Updating files:  56% (1706/3046)
Updating files:  57% (1737/3046)
Updating files:  58% (1767/3046)
Updating files:  59% (1798/3046)
Updating files:  60% (1828/3046)
Updating files:  61% (1859/3046)
Updating files:  62% (1889/3046)
Updating files:  63% (1919/3046)
Updating files:  64% (1950/3046)
Updating files:  65% (1980/3046)
Updating files:  6

In [40]:
CUSTOM_MODEL_NAME = 'my_ssd_mobnet'

In [6]:
# create a folder to store the pretraiined model's config file
!mkdir {'Tensorflow\workspace\models\\'+CUSTOM_MODEL_NAME}

A subdirectory or file Tensorflow\workspace\models\my_ssd_mobnet already exists.


In [ ]:
# Can download a pretrained model from the belo given link . I downloaded the "SSD MobileNet v2 320x320" model
# https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md
# Once download is done copy the pipeline.config file and paste it in the folder created above

In [4]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format
# The Tensorflow Object Detection API uses Protobufs to configure model and training parameters. 
# Before the framework can be used, the Protobuf libraries must be downloaded and compiled.

In [41]:
# Setting the path of configuration file and grabbing the content and fine tune the model to us as per our need
CONFIG_PATH = MODEL_PATH+'/'+CUSTOM_MODEL_NAME+'/pipeline.config'

In [9]:
config = config_util.get_configs_from_pipeline_file(CONFIG_PATH)

In [27]:
config

{'model': ssd {
   num_classes: 90
   image_resizer {
     fixed_shape_resizer {
       height: 320
       width: 320
     }
   }
   feature_extractor {
     type: "ssd_mobilenet_v2_fpn_keras"
     depth_multiplier: 1.0
     min_depth: 16
     conv_hyperparams {
       regularizer {
         l2_regularizer {
           weight: 4e-05
         }
       }
       initializer {
         random_normal_initializer {
           mean: 0.0
           stddev: 0.01
         }
       }
       activation: RELU_6
       batch_norm {
         decay: 0.997
         scale: true
         epsilon: 0.001
       }
     }
     use_depthwise: true
     override_base_feature_extractor_hyperparams: true
     fpn {
       min_level: 3
       max_level: 7
       additional_layer_depth: 128
     }
   }
   box_coder {
     faster_rcnn_box_coder {
       y_scale: 10.0
       x_scale: 10.0
       height_scale: 5.0
       width_scale: 5.0
     }
   }
   matcher {
     argmax_matcher {
       matched_threshold: 0.5
   

In [13]:
# Copying the values from the file and updating the values
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(CONFIG_PATH, "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  

In [8]:
# For fine tuning the model we will update the following values
pipeline_config.model.ssd.num_classes = 7  #changing the no. of classes to 7 as per our requirement
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = PRETRAINED_MODEL_PATH+'/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0' #the point where we want our model tp fran from
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/train.record']
pipeline_config.eval_input_reader[0].label_map_path = ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/test.record']

NameError: name 'pipeline_config' is not defined

In [15]:
# Updating the values in the original file
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(CONFIG_PATH, "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)   

In [10]:
print("""python {}/research/object_detection/model_main_tf2.py --model_dir={}/{} --pipeline_config_path={}/{}/pipeline.config --num_train_steps=3000""".format(APIMODEL_PATH, MODEL_PATH,CUSTOM_MODEL_NAME,MODEL_PATH,CUSTOM_MODEL_NAME))
# Copy the o/p and run in the cmd

python Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=Tensorflow/workspace/models/my_ssd_mobnet --pipeline_config_path=Tensorflow/workspace/models/my_ssd_mobnet/pipeline.config --num_train_steps=3000


# Real Time Sign Detection

In [6]:
import os
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

In [7]:
configs = config_util.get_configs_from_pipeline_file(CONFIG_PATH)
detection_model = model_builder.build(model_config=configs['model'],is_training=False)

ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(CHECKPOINT_PATH,'ckpt-4')).expect_partial() #change checkpoint value

# Tensorflow Function to make predictions
@tf.function
def detect_fn(image):
    image,shapes=detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image,shapes)
    detections=detection_model.postprocess(prediction_dict,shapes)
    return detections

In [67]:
import cv2
import numpy as np

In [68]:
category_index = label_map_util.create_category_index_from_labelmap(ANNOTATION_PATH+'/label_map.pbtxt')

In [33]:
cap = cv2.VideoCapture(0)
width=int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height=int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

In [34]:
# Code for real time detection 
while True: 
    ret, frame = cap.read()
    image_np = np.array(frame)
    
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)
    
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=5,
                min_score_thresh=.5,
                agnostic_mode=False)

    cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (800, 600)))
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        break

In [19]:
cap.release()
cv2.destroyAllWindows()

NameError: name 'cap' is not defined


# Test Model on Images 

In [17]:
import cv2 
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

import os
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

cv2.destroyAllWindows()

In [18]:
category_index = label_map_util.create_category_index_from_labelmap(ANNOTATION_PATH+'/label_map.pbtxt')

In [35]:
IMAGE_PATH='Tensorflow/workspace/images'
IMAGE_PATH = os.path.join(IMAGE_PATH, 'test', 'WIN_20220726_16_06_38_Pro.jpg')

In [29]:
configs = config_util.get_configs_from_pipeline_file(CONFIG_PATH)
detection_model = model_builder.build(model_config=configs['model'],is_training=False)

ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(CHECKPOINT_PATH,'ckpt-4')).expect_partial()

# Tensorflow Function to make predictions
@tf.function
def detect_fn(image):
    image,shapes=detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image,shapes)
    detections=detection_model.postprocess(prediction_dict,shapes)
    return detections

In [1]:
img = cv2.imread(IMAGE_PATH)
image_np = np.array(img)

input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
detections = detect_fn(input_tensor)


num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy()
              for key, value in detections.items()}
detections['num_detections'] = num_detections


# detection_classes should be ints.
detections['detection_classes'] = detections['detection_classes'].astype(np.int64)


label_id_offset = 1
image_np_with_detections = image_np.copy()

viz_utils.visualize_boxes_and_labels_on_image_array(
            image_np_with_detections,
            detections['detection_boxes'],
            detections['detection_classes']+label_id_offset,
            detections['detection_scores'],
            category_index,
            use_normalized_coordinates=True,
            max_boxes_to_draw=5,
            min_score_thresh=.8,
            agnostic_mode=False)
# print(detections)
plt.imshow(cv2.cvtColor(image_np_with_detections, cv2.COLOR_BGR2RGB))
plt.show()